In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Extracting file to C:\Users\10838233\.dgl\cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [3]:
g = dataset[0]


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
options = {
    'node_color': 'black',
    'node_size': 20,
    'width': 1,
}
G = dgl.to_networkx(g)
plt.figure(figsize=[15,7])
nx.draw(G, **options)

In [4]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([False, False, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False]), 'test_mask': tensor([ True,  True, False,  ..., False, False, False]), 'label': tensor([4, 4, 4,  ..., 4, 3, 3]), 'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]])}
Edge features
{'__orig__': tensor([  298,  9199,  1153,  ..., 10415,  5255,  6356])}


In [5]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [6]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

C:\Users\10838233\Anaconda3\envs\tf2\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 1.946, val acc: 0.218 (best 0.218), test acc: 0.198 (best 0.198)
In epoch 5, loss: 1.900, val acc: 0.534 (best 0.534), test acc: 0.567 (best 0.567)
In epoch 10, loss: 1.828, val acc: 0.598 (best 0.598), test acc: 0.600 (best 0.600)
In epoch 15, loss: 1.730, val acc: 0.640 (best 0.644), test acc: 0.657 (best 0.653)
In epoch 20, loss: 1.607, val acc: 0.664 (best 0.664), test acc: 0.671 (best 0.671)
In epoch 25, loss: 1.459, val acc: 0.700 (best 0.700), test acc: 0.704 (best 0.704)
In epoch 30, loss: 1.291, val acc: 0.716 (best 0.716), test acc: 0.721 (best 0.716)
In epoch 35, loss: 1.112, val acc: 0.722 (best 0.724), test acc: 0.726 (best 0.722)
In epoch 40, loss: 0.931, val acc: 0.728 (best 0.728), test acc: 0.736 (best 0.736)
In epoch 45, loss: 0.762, val acc: 0.746 (best 0.746), test acc: 0.746 (best 0.746)
In epoch 50, loss: 0.612, val acc: 0.756 (best 0.756), test acc: 0.757 (best 0.754)
In epoch 55, loss: 0.487, val acc: 0.762 (best 0.762), test acc: 0.763 (best 0